In [4]:
# -*- coding: utf-8 -*-

import openpyxl
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from numpy.linalg import inv
from scipy.stats import norm
import pandas as pd


def QQ(vec):
    v = np.sort(vec)
    q = stats.norm.ppf((np.arange(len(v))+1-0.375)/(len(v)+0.25))
    return v, q

def RB_fit(RB):
    #RB_raw = np.array(RB)
    RB_ord = np.sort(RB)/100
    th_q = stats.norm.ppf((np.arange(len(RB))+1-0.375)/(len(RB)+0.25))
    q = th_q
    l = np.log(RB_ord)
    h = np.log(1-RB_ord)
    d = (np.mean(l*l) - pow(np.mean(l),2))*(np.mean(h*h) - pow(np.mean(h),2)) - pow(np.mean(l*h) - np.mean(l)*np.mean(h),2)
    d1 = (np.mean(h*h) - pow(np.mean(h),2))*(np.mean(l*q) - np.mean(l)*np.mean(q))-(np.mean(l*h) - np.mean(l)*np.mean(h))*(np.mean(h*q) - np.mean(h)*np.mean(q))
    d2 = (np.mean(l*h) - np.mean(l)*np.mean(h))*(np.mean(l*q) - np.mean(l)*np.mean(q)) - (np.mean(l*l) - pow(np.mean(l),2))*(np.mean(h*q) - np.mean(h)*np.mean(q))
    if d1 < 0:
        a = 0
        b = -(np.mean(h*q) - np.mean(h)*np.mean(q))/(np.mean(h*h) - pow(np.mean(h),2))
    elif d2 < 0:
        a = (np.mean(l*q) - np.mean(l)*np.mean(q))/(np.mean(l*l) - pow(np.mean(l),2))
        b = 0
    else:
        a = d1/d
        b = d2/d
#     a = d1/d
#     b = d2/d
    c = np.mean(q) - a*np.mean(l) + b*np.mean(h)
    RB_t = a*np.log(RB/100) - b*np.log(1-RB/100) + c
    _, pv = stats.shapiro(RB_t)
    return a, b, c, pv, RB_t, th_q

def TP_fit(TP):
    TP_ord = np.sort(TP)
    th_q = stats.norm.ppf((np.arange(len(TP))+1-0.375)/(len(TP)+0.25))
    q = th_q
    g = np.log(TP_ord)
    t = TP_ord
    d = (np.mean(g*g) - pow(np.mean(g),2))*(np.mean(t*t) - pow(np.mean(t),2)) - pow(np.mean(g*t) - np.mean(g)*np.mean(t),2)
    d1 = (np.mean(t*t) - pow(np.mean(t),2))*(np.mean(g*q) - np.mean(g)*np.mean(q))-(np.mean(g*t) - np.mean(g)*np.mean(t))*(np.mean(t*q) - np.mean(t)*np.mean(q))
    d2 = (np.mean(g*g) - pow(np.mean(g),2))*(np.mean(t*q) - np.mean(t)*np.mean(q))-(np.mean(t*g) - np.mean(t)*np.mean(g))*(np.mean(g*q) - np.mean(g)*np.mean(q))
    if d1 < 0:
        a = 0
        b = (np.mean(t*q) - np.mean(t)*np.mean(q))/(np.mean(t*t) - pow(np.mean(t),2))
    elif d2 < 0:
        a = (np.mean(g*q) - np.mean(g)*np.mean(q))/(np.mean(g*g) - pow(np.mean(g),2))
        b = 0
    else:
        a = d1/d
        b = d2/d
#     a = d1/d
#     b = d2/d
    c = np.mean(q) - a*np.mean(g) - b*np.mean(t)
    TP_t = a*np.log(TP) + b*TP + c
    _, pv = stats.shapiro(TP_t)
    return a, b, c, pv, TP_t, th_q

def Roystontest(n, k, Data):
    x = np.log(n)
    gamma = 0
    mu = -1.5861 - 0.31082*x - 0.083751*pow(x,2) + 0.0038915*pow(x,3)
    sigma = np.exp(-0.4803 -0.082676*x + 0.0030302*pow(x,2))
    W1 = stats.shapiro(Data[:,0])[0]
    W2 = stats.shapiro(Data[:,1])[0]
    Z1 = (np.log(1-W1)-mu)/sigma   
    Z2 = (np.log(1-W2)-mu)/sigma
    G1 = stats.norm.ppf(stats.norm.cdf(-Z1)/2)**2
    G2 = stats.norm.ppf(stats.norm.cdf(-Z2)/2)**2
    u = 0.715
    v = 0.21364+0.015124*pow(x,2)-0.0018034*pow(x,3)
    la = 5
    corr_data = np.corrcoef(Data[:,0],Data[:,1])
    new_corr = pow(corr_data,la) * (1 - u*pow(1 - corr_data,u)/v)
    total_corr = np.sum(new_corr) - k
    avg_corr = total_corr/(k**2-k)
    est_e = k/(1 + (k - 1) * avg_corr)
    Roys = est_e*(G1+G2)/k
    p_value = 1- stats.chi2.cdf(Roys, df = est_e)
    return p_value

def rntiprob(length, Data):
    rntiprob = np.zeros(length)   #rnti의 비율 계산
    rntiprobsum = np.zeros(length)   # rnti 비율 누적
    for i in range(length):
        rntiprob[i] = len(Data[Data[:,2] == (i+3),2])/len(Data[:,2])
    for i in range(length):
        rntiprobsum[i] = sum(rntiprob[0:i+1])
    return rntiprobsum


def fixRNTI(rntiprobsum):
    a = np.random.uniform(0,1)
    i = 0
    while rntiprobsum[i] < a:
        i = i+1
    return i+3

def SamplingperRnti(Data, fixed_RNTI):
    RB_sample = Data[Data[:,2] == fixed_RNTI, 1]   # RNTI = N 인 RB,RNTI,TP 
    TP_sample = Data[Data[:,2] == fixed_RNTI, 3]
    sampling_size = min(30, int(len(RB_sample)/2))
    sample_index = np.random.choice(len(RB_sample),sampling_size, replace = False)
    RB_sample = RB_sample[sample_index,]
    TP_sample = TP_sample[sample_index,]
    return(RB_sample, TP_sample)

def FindParams(fixed_rnti, Data):
    RB_rnd_sample, TP_rnd_sample = SamplingperRnti(Data, fixed_rnti)   # random한 RNTI에 대해 RB, TP random sampling
    a_RB, b_RB, c_RB, pv_RB, RB_trans_data, tq_RB = RB_fit(RB_rnd_sample)
    a_TP, b_TP, c_TP, pv_TP, TP_trans_data, tq_TP = TP_fit(TP_rnd_sample)
    total_trans_data = np.hstack((RB_trans_data.reshape(-1,1), TP_trans_data.reshape(-1,1)))   # transformed data로 total data 생성
    cov = np.cov(total_trans_data.T)[0,1]
    #corr = np.corrcoef(total_trans_data.T)[0,1]
    joint_pv_R = Roystontest(len(TP_trans_data), 2, total_trans_data)
    se = np.mean(TP_rnd_sample*0.001/(RB_rnd_sample*2/100*10))
    if (pv_RB > 0.05)& (pv_TP >0.05) & (joint_pv_R > 0.05):
        return np.array([fixed_rnti, a_RB, b_RB, c_RB, a_TP, b_TP, c_TP, np.mean(total_trans_data), cov, se])
    else:
        return [0]
    


In [14]:
def load_data():
    RD = openpyxl.load_workbook('data_prep_ver2.xlsx', data_only=True)
    #st = RD['Sheet2']   #band 20MHz
    st = RD['Sheet1']   #band 10MHz
    Data = np.zeros((1,5))
    i = 2
    value = st['B%d' %i].value
    while value != None:
        Data = np.vstack((Data, np.array([st['B%d' %i].value, st['C%d' % i].value, st['D%d' %i].value, st['E%d' %i].value, st['F%d' %i].value])))
        # PCI, RB, RNTI, TP, Timestamp
        i += 1
        value = st['B%d' %i].value
    Data = Data[1:,:].astype(np.float64)
    len_per_rnti = np.array([len(Data[Data[:,2] == np.float(i), 1]) for i in range(20)])
    
    #length가 30 미만인 RNTI 삭제
    selected_rnti = np.where(len_per_rnti>30)
    Data = Data[np.where(np.isin(Data[:,2], selected_rnti)),]
    Data = np.reshape(Data, (len(Data[0]),5))
    rnti_array = np.array(selected_rnti)
    rnti_array = rnti_array.reshape(-1)

    rntiprobsum = rntiprob(len(rnti_array), Data)
    return Data, rntiprobsum

    

In [18]:
def main():
    Data, rntiprobsum = load_data()
    ParamArray = np.zeros((10))
    sample_size =10000
    i = 0
    transtp10 = []
    while i < sample_size:
        print(i)
        fixed_rnti = fixRNTI(rntiprobsum)
        #fixed_rnti = selected_rnti[np.random.randint(len(selected_rnti))]
        params = FindParams(fixed_rnti, Data)
        print(params)
        if len(params) == 10:
            ParamArray = np.vstack((ParamArray, params))
            i += 1
        else:
            continue
    print("end")
    return ParamArray

if __name__ == "__main__":
    main()
    

0
[ 3.00000000e+00  4.06542897e-01  1.46541799e+00  6.47476853e-01
  5.70094380e-01  4.06107657e-05 -3.87391483e+00 -3.36767651e-16
  7.53717515e-01  6.04452828e-01]
1
[ 3.00000000e+00  3.78059872e-01  1.56741277e+00  3.85912766e-01
  3.94319818e-01  1.92631765e-04 -3.18888400e+00 -2.88657986e-16
  5.79402349e-01  7.30509725e-01]
2
[ 4.00000000e+00  5.15098352e-01  4.68016467e-01  1.24990230e+00
  5.11812201e-01  1.77929349e-05 -3.42787368e+00  2.03540888e-17
  7.92708602e-01  7.18148828e-01]
3
[ 1.00000000e+01  1.01040975e+00  8.30879195e-01  1.38500768e+00
  1.18915146e+00  5.57799422e-05 -1.01460545e+01 -1.15463195e-15
  5.43862527e-01  1.29222695e+00]
4
[ 3.00000000e+00  3.74969235e-01  3.30496162e+00 -1.24821622e-01
  4.77083963e-01  1.32751345e-04 -4.00723961e+00 -6.03221177e-16
  6.30869188e-01  7.85929822e-01]
5
[ 4.00000000e+00  4.87968175e-01  5.24070453e-01  9.33507342e-01
  5.02691860e-01  9.90481306e-05 -3.75157687e+00  1.11022302e-16
  8.01721956e-01  7.33401616e-01]
6
[ 

[ 5.00000000e+00  4.64774980e-01  9.71572133e-01  5.74478831e-01
  4.39816966e-01  1.30836008e-04 -3.83080514e+00 -8.14163551e-17
  6.64240025e-01  9.02654145e-01]
77
[ 4.00000000e+00  3.56265162e-01  1.50292506e+00  1.65387423e-01
  4.26169223e-01  1.72280609e-04 -3.58568974e+00 -7.03141249e-17
  5.54391856e-01  6.83274101e-01]
78
[ 3.00000000e+00  3.47230278e-01  1.61863722e+00  2.22757070e-01
  4.12951034e-01  7.13281491e-05 -3.13893095e+00  1.48029737e-16
  6.68316539e-01  7.28604991e-01]
79
[ 4.00000000e+00  3.59876715e-01  1.16173333e+00  3.57850709e-01
  4.41312038e-01  3.84099681e-05 -3.36758609e+00 -3.60822483e-16
  7.46902579e-01  8.53520688e-01]
80
[ 5.00000000e+00  4.17687937e-01  1.06396599e+00  7.59386471e-01
  3.99713169e-01  6.02819606e-05 -3.08050652e+00  3.46019509e-16
  8.00174077e-01  9.48463421e-01]
81
[ 3.00000000e+00  3.14438946e-01  2.00166545e+00 -1.13530395e-01
  5.07565694e-01  4.58025010e-05 -3.90736619e+00  4.10782519e-16
  6.25615187e-01  7.12248331e-01]
8

  6.50895028e-01  1.12488082e+00]
151
[ 6.00000000e+00  5.27075084e-01  9.29082883e-01  7.85508337e-01
  6.38719042e-01  2.92990250e-05 -4.97130377e+00  2.03540888e-17
  7.52481390e-01  9.73193055e-01]
152
[ 8.00000000e+00  7.61694311e-01  1.14864283e+00  1.13961155e+00
  3.45996858e-01  3.50069315e-04 -3.90611807e+00  2.07241631e-16
  4.86833188e-01  1.07189761e+00]
153
[ 3.00000000e+00  3.38238970e-01  1.86265262e+00 -7.73897261e-02
  1.64877156e-01  3.54492929e-04 -2.35542813e+00 -2.07241631e-16
  5.35518917e-01  7.11105291e-01]
154
[ 3.00000000e+00  6.24395883e-01  5.76599601e-01  1.14505455e+00
  3.98201982e-01  1.39899551e-04 -3.26922653e+00 -2.81256500e-16
  6.48129698e-01  8.30909256e-01]
155
[ 3.00000000e+00  3.65481466e-01  1.45827032e+00  3.95263315e-01
  4.78266995e-01  3.37684949e-05 -3.45367514e+00 -2.03540888e-16
  7.89894758e-01  6.84518354e-01]
156
[ 5.00000000e+00  3.49008510e-01  1.23462848e+00  4.98588031e-01
  3.76944358e-01  7.44897939e-05 -2.91887850e+00  1.55431

[ 3.00000000e+00  1.98763431e-01  4.12577931e+00 -3.75585444e-01
  3.71979288e-01  1.04976505e-04 -2.93660725e+00 -2.15568304e-16
  6.45473165e-01  9.09863979e-01]
226
[ 4.00000000e+00  6.49041953e-01  6.58393978e-01  1.13299582e+00
  7.09097046e-01  3.65142984e-05 -4.98425346e+00 -2.73855013e-16
  7.16311039e-01  5.20734754e-01]
227
[ 4.00000000e+00  3.96919829e-01  2.39510630e+00  6.16840951e-01
  5.01599531e-01  4.78974552e-05 -3.36407617e+00  2.14643118e-16
  7.40302704e-01  6.96956926e-01]
228
[ 5.00000000e+00  6.21155466e-01  8.96908174e-01  1.05391594e+00
  4.61227326e-01  9.90709379e-05 -3.78710582e+00  2.66453526e-16
  5.95765023e-01  9.73751692e-01]
229
[ 4.00000000e+00  4.41456513e-01  7.38526214e-01  4.95342950e-01
  4.98322071e-01  1.09921636e-04 -4.00744568e+00 -1.62832710e-16
  6.76583489e-01  7.04911217e-01]
230
[ 4.00000000e+00  4.22718266e-01  1.54472661e+00  6.08915173e-01
  4.44968584e-01  1.11922685e-04 -3.25433593e+00  1.33226763e-16
  7.31937252e-01  6.32538789e-

[ 4.00000000e+00  4.99005394e-01  5.98577363e-01  9.28246534e-01
  6.89678689e-01  1.40351361e-05 -4.73838466e+00 -2.14643118e-16
  8.08241468e-01  6.00295004e-01]
301
[ 5.00000000e+00  3.36529680e-01  1.73251951e+00  6.35127737e-02
  4.66759855e-01  6.32466290e-05 -3.78482319e+00  1.83186799e-16
  6.76136861e-01  8.68701230e-01]
302
[ 4.00000000e+00  3.79996132e-01  1.58580987e+00  4.71367384e-01
  4.87212153e-01  3.50897514e-05 -3.45961416e+00 -8.88178420e-17
  8.25983153e-01  6.54571932e-01]
303
[ 8.00000000e+00  6.18586686e-01  8.31287518e-01  8.93869991e-01
  4.84665453e-01  1.01046708e-04 -4.20932115e+00 -1.11022302e-17
  6.84041775e-01  1.02243227e+00]
304
[ 3.00000000e+00  3.86427442e-01  1.08073623e+00  4.36345032e-01
  3.66549232e-01  8.77287900e-05 -2.91577324e+00 -1.90588286e-16
  7.21935430e-01  7.28072465e-01]
305
[ 3.00000000e+00  4.96131905e-01  5.94707597e-01  8.20056170e-01
  3.34266448e-01  2.31434351e-04 -2.88452095e+00 -1.03620816e-16
  6.87728120e-01  5.92457807e-

...

[ 1.00000000e+01  6.84230157e-01  1.74283209e+00  5.44929912e-01
  1.06182531e+00  5.17221610e-05 -9.20269112e+00 -1.11022302e-17
  7.12770010e-01  1.32275539e+00]
2189
[ 4.00000000e+00  3.79417829e-01  1.16296143e+00  3.07394113e-01
  5.14364679e-01  4.96854371e-05 -3.84656226e+00  1.29526020e-16
  8.06263928e-01  5.79375398e-01]
2190
[ 4.00000000e+00  5.20693568e-01  7.16869718e-01  1.08374609e+00
  3.85570182e-01  2.71802707e-04 -3.13800917e+00  1.85037171e-17
  6.54013067e-01  9.07024341e-01]
2191
[ 4.00000000e+00  6.21615701e-01  7.45684399e-01  1.50271652e+00
  4.89062799e-01  1.24600922e-04 -3.50586510e+00 -5.92118946e-16
  6.96654246e-01  8.27070722e-01]
2192
[ 3.00000000e+00  4.78541505e-01  1.25173951e+00  6.85882128e-01
  5.32474999e-01  2.18552151e-05 -3.81518464e+00 -5.18104078e-17
  6.79194422e-01  8.26677105e-01]
2193
[ 6.00000000e+00  4.62267206e-01  8.66771240e-01  6.78089398e-01
  6.88842380e-01  1.04212930e-05 -5.14692390e+00  5.07001848e-16
  7.46007227e-01  9.04541

[ 5.00000000e+00  5.50880317e-01  4.16628737e-01  1.07916511e+00
  3.35842592e-01  2.43252257e-04 -2.97215403e+00 -7.40148683e-18
  6.35942452e-01  8.36898351e-01]
2260
[ 4.00000000e+00  3.71214069e-01  7.96802138e-01  6.24865595e-01
  4.44523075e-01  9.54604292e-05 -3.49914851e+00 -3.77475828e-16
  7.38725141e-01  1.04163023e+00]
2261
[0]
2261
[ 9.00000000e+00  5.39423109e-01  1.77901395e+00  4.91197099e-01
  5.80037953e-01  1.31516094e-04 -5.13763372e+00  6.14892752e-16
  6.43339061e-01  1.08706359e+00]
2262
[ 4.00000000e+00  5.05223483e-01  3.15983732e+00  3.15410401e-01
  3.78898373e-01  1.43424781e-04 -3.30834663e+00 -1.22124533e-16
  5.91821406e-01  9.14821192e-01]
2263
[ 3.00000000e+00  5.26100976e-01  8.92020013e-01  9.88006871e-01
  4.69943382e-01  7.21694840e-05 -3.44752988e+00  4.81096644e-17
  7.21965238e-01  7.32628135e-01]
2264
[ 5.00000000e+00  6.07464695e-01  1.14188485e+00  1.08545364e+00
  6.01609349e-01  4.18946150e-05 -4.52687292e+00  1.60982339e-16
  5.95103523e-01

[ 4.00000000e+00  6.08758531e-01  1.14700184e+00  8.32808102e-01
  7.75838403e-01  2.68952729e-05 -5.92550664e+00 -5.84717460e-16
  6.69312573e-01  8.96214058e-01]
2336
[ 3.00000000e+00  3.89118043e-01  3.30900524e+00 -1.02200805e-01
  7.31208709e-01  6.86699018e-05 -5.48113587e+00 -2.20194233e-16
  6.12573551e-01  5.94993003e-01]
2337
[ 3.00000000e+00  3.80362218e-01  2.31360075e+00  7.52072252e-01
  3.92877375e-01  1.69982523e-04 -2.60295395e+00  1.14723046e-16
  7.47948877e-01  5.80391952e-01]
2338
[ 4.00000000e+00  7.78772691e-01  3.94693352e-01  1.24525566e+00
  5.75798280e-01  7.79695152e-05 -4.58965471e+00 -3.70074342e-17
  7.28064597e-01  7.04030874e-01]
2339
[ 6.00000000e+00  4.17271393e-01  2.16561756e+00  4.51414105e-01
  4.44567628e-01  4.60854540e-05 -3.57582651e+00 -1.70234197e-16
  7.64720535e-01  1.19724122e+00]
2340
[ 3.00000000e+00  3.75325442e-01  2.49986437e+00  3.00731516e-01
  3.39596201e-01  1.81945854e-04 -2.91193574e+00 -3.44169138e-16
  6.55199238e-01  8.42752

[ 3.00000000e+00  4.23007179e-01  8.71494863e-01  1.01858968e+00
  2.79582936e-01  3.26605974e-04 -2.18222358e+00 -2.14643118e-16
  8.21327717e-01  5.65135188e-01]
2418
[ 4.00000000e+00  5.21048385e-01  5.93713912e-01  9.42888079e-01
  4.28572490e-01  1.26934952e-04 -3.30121961e+00  1.70234197e-16
  6.85897812e-01  7.03031854e-01]
2419
[ 6.00000000e+00  5.82897671e-01  7.27100206e-01  9.51116589e-01
  6.52453645e-01  1.15018252e-05 -4.94032647e+00  1.11022302e-17
  7.03018341e-01  9.18981221e-01]
2420
[ 3.00000000e+00  4.52119378e-01  1.84347411e+00  8.85422440e-01
  5.16191499e-01  2.57580304e-05 -3.45711819e+00  2.27595720e-16
  7.58201855e-01  7.51725472e-01]
2421
[ 1.00000000e+01  1.30166375e+00  4.01252453e-01  2.39287941e+00
  1.46693450e+00  0.00000000e+00 -1.20421926e+01  3.44169138e-16
  5.84008196e-01  1.47387347e+00]
2422
[ 5.00000000e+00  4.86732433e-01  1.49904270e+00  3.08115045e-01
  5.65176599e-01  1.13551515e-04 -4.59722345e+00  2.31296463e-16
  6.67990919e-01  6.66762

KeyboardInterrupt: 

In [ ]:
raw_data_20 = ParamArray[1:,:]